In [ ]:
!git clone https://github.com/Srija616/DeepLearning_Assignment_2.git

In [ ]:
!pip install wandb
import wandb

In [1]:
import os
import wandb
import random
import numpy as np

import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset, random_split, Subset
import torchvision
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchmetrics import Accuracy
torch.manual_seed(42)

In [2]:
path = os.path.dirname(os.path.realpath('__file__'))
train_path = os.path.join(path, 'inaturalist_12K/train/')
test_path = os.path.join(path, 'inaturalist_12K/val/')
print (train_path, test_path)

/home/vg/ttsteam/repos/srija/inaturalist_12K/train/ /home/vg/ttsteam/repos/srija/inaturalist_12K/val/


In [7]:
def preprocess_data(train_path, test_path, data_aug=False):
    if data_aug == True:
        transforms = T.Compose([T.RandomResizedCrop((224, 224)),
                            T.RandomHorizontalFlip(p=0.5),
                            T.RandomVerticalFlip(p=0.5),
                            T.RandomRotation(degrees=45),
                            T.ToTensor(),
                            T.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
                                
                ])
    else:
        transforms = T.Compose([T.RandomResizedCrop((224, 224)),
                    T.ToTensor(),
                    T.Normalize(mean=[0, 0, 0], std=[1, 1, 1])])

    dataset = ImageFolder(train_path, transform=transforms)
    test_dataset = ImageFolder(test_path, transform=transforms)

    train_size = int(0.8*len(dataset))
    val_size = (len(dataset) - train_size)

    class_labels = [label for _, label in dataset]
    target = np.array(class_labels)

    from sklearn.model_selection import StratifiedShuffleSplit
    stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

    for train_idx, val_idx in stratified_splitter.split(np.zeros(len(target)), target):
        train_dataset = Subset(dataset, train_idx)
        val_dataset = Subset(dataset, val_idx)

    return train_dataset, val_dataset, test_dataset

In [8]:
import sys
sys.path.append("/home/vg/ttsteam/repos/srija/DeepLearning_Assignment_2/models")
from model_partb import FinetuneImgModel
import pytorch_lightning as pl

In [9]:
config = {
    'data_aug':True
}

In [12]:
def finetuneimgmodel():
  train_dataset, val_dataset, test_dataset = preprocess_data(train_path, test_path, config["data_aug"])
  model = FinetuneImgModel(
      train_dataset = train_dataset,
      test_dataset = test_dataset,
      val_dataset = val_dataset 
  )
  
  logger = pl.loggers.WandbLogger(name="resnet50-finetuning-2.0", project="DL-Assignment2", entity="srija17199")
  logger.watch(model)
  callbacks = [pl.callbacks.ModelCheckpoint(monitor='val_acc', mode='max', save_top_k=1, save_last=True)]
  trainer = pl.Trainer(max_epochs=35, devices=-1, logger=logger, callbacks=callbacks, precision='bf16-mixed')
  trainer.fit(model)

In [13]:
finetuneimgmodel()

/home/vg/.miniconda3/envs/dl_ass2/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/vg/.miniconda3/envs/dl_ass2/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory ./DL-Assignment2/ruc0zkq5/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type               | Params
------------------------------------------------
0 | fc1      | Sequential         | 23.5 M
1 | output   | Linear             | 20.5 K
2 | accuracy | Multicl

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=35` reached.
